# The Piecewise-Parabolic Method

This notebook documents the function from the original `GiRaFFE` that calculates the flux for $\tilde{S}_i$ according to the  piecewise-parabolic method (PPM) of [Colella and Woodward (1984)](https://crd.lbl.gov/assets/pubs_presos/AMCS/ANAG/A141984.pdf). 

The differential equations that `GiRaFFE` evolves have two different terms that contribute to the time evolution of some quantity: the flux term and the source term. The PPM method is what the original `GiRaFFE` uses to handle the flux term; hopefully, using this instead of finite-differencing will fix some of the problems we've been having with `GiRaFFE_HO`.

This algorithm is not quite as accessible as the much simpler finite-difference methods; as such, [this notebook](https://mybinder.org/v2/gh/python-hydro/how_to_write_a_hydro_code/master) is recommended as an introduction. It covers a simpler reconstruction scheme, and proved useful in preparing the documentation for this more complicated scheme.

The algorithm for finite-volume methods in general is as follows: 

1. The Reconstruction Step
    1. Within each cell, fit to a function that conserves the volume in that cell using information from the neighboring cells
        * For PPM, we will naturally use parabolas
    1. Use that fit to define the state at the left and right interface of each cell
    1. Apply a slope limiter to mitigate Gibbs phenomenon
1. **Solving the Riemann Problem (This notebook, $\tilde{S}_i$ only)**
    1. **Use the left and right reconstructed states to calculate the unique state at boundary**
    1. **Use the unique state to estimate the derivative in the cell**
1. Repeat the above for each conservative gridfunction in each direction

<a id='toc'></a>

# Table of Contents
$$\label{toc}$$

This module is organized as follows

0. [Step 0](#prelim): Preliminaries
1. [Step 1](#s_i_flux): The $\tilde{S}_i$ function
    1. [Step 1.a](#define): The function declaration
    1. [Step 1.b](#speed_limit): Speed-limit the velocities
    1. [Step 1.c](#smallb): Magnetic field in the comoving fluid frame
    1. [Step 1.d](#hydro_speed): Hydrodynamic wave speeds
    1. [Step 1.e](#useful_quantities): Some final needed quantities
    1. [Step 1.f](#fluxes): Compute the fluxes
1. [Step 2](#latex_pdf_output): Output this module to $\LaTeX$-formatted PDF file


<a id='prelim'></a>

## Step 0: Preliminaries \[Back to [top](#toc)\]
$$\label{prelim}$$

This first block of code just sets up a subdirectory within `GiRaFFE_standalone_Ccodes/` to which we will write the C code. We will also import the core NRPy+ functionality and register the needed gridfunctions. Doing so will let NRPy+ figure out where to read and write data from/to. 

In [1]:
import cmdline_helper as cmd
import os
outdir = "GiRaFFE_standalone_Ccodes/PPM"
cmd.mkdir(os.path.join(outdir,"/"))

import NRPy_param_funcs as par
import indexedexp as ixp
import grid as gri
import finite_difference as fin
from outputC import *

par.set_parval_from_str("grid::DIM", 3)
DIM = par.parval_from_str("grid::DIM")

# These are the standard gridfunctions we've used before.
gammaDD = ixp.register_gridfunctions_for_single_rank2("AUXEVOL","gammaDD", "sym01",DIM=3)
betaU   = ixp.register_gridfunctions_for_single_rank1("AUXEVOL","betaU",DIM=3)
alpha   = gri.register_gridfunctions("AUXEVOL","alpha")
ValenciavU = ixp.register_gridfunctions_for_single_rank1("AUXEVOL","ValenciavU",DIM=3)
AD = ixp.register_gridfunctions_for_single_rank1("EVOL","AD",DIM=3)
BU = ixp.register_gridfunctions_for_single_rank1("AUXEVOL","BU",DIM=3)
gammaUU = ixp.register_gridfunctions_for_single_rank2("AUXEVOL","gammaDD", "sym01",DIM=3)
gammadet = ixp.gri.register_gridfunctions("AUXEVOL","gammadet")

# We'll need some more gridfunctions, now, to represent the reconstructions of BU and ValenciavU
Valenciav_rU = ixp.register_gridfunctions_for_single_rank1("AUXEVOL","Valenciav_rU",DIM=3)
B_rU = ixp.register_gridfunctions_for_single_rank1("AUXEVOL","B_rU",DIM=3)
Valenciav_lU = ixp.register_gridfunctions_for_single_rank1("AUXEVOL","Valenciav_lU",DIM=3)
B_lU = ixp.register_gridfunctions_for_single_rank1("AUXEVOL","B_lU",DIM=3)

Stilde_rhsD = ixp.zerorank1("Stilde_rhsD",DIM=3)

Error: Tried to register the gridfunction "gammaDD00" twice (ignored type)




SystemExit: 1

/home/penelson/juppp/site-packages/IPython/core/interactiveshell.py:2886: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In GRFFE, the evolution equation for the Poynting flux $\tilde{S}_i$ is given as 
$$
\partial_t \tilde{S}_i + \partial_j \left( \alpha \sqrt{\gamma} T^j_{{\rm EM} i} \right) = \frac{1}{2} \alpha \sqrt{\gamma} T^{\mu \nu}_{\rm EM} \partial_i g_{\mu \nu}.
$$
We can then see that, if we rewrite this, the right-hand side (RHS) describing the time evolution $\partial_t \tilde{S}_i$ consists of two terms:
$$
\partial_t \tilde{S}_i = \underbrace{- \partial_j \left( \alpha \sqrt{\gamma} T^j_{{\rm EM} i} \right)}_{\rm Flux\ term} + \underbrace{\frac{1}{2} \alpha \sqrt{\gamma} T^{\mu \nu}_{\rm EM} \partial_i g_{\mu \nu}}_{\rm Source\ term}.
$$

It is the flux term that concerns us here; the following function will compute its value and add it to $\partial_t \tilde{S}_i$.

<a id='s_i_flux'></a>

## Step 1: The $\tilde{S}_i$ function \[Back to [top](#toc)\]
$$\label{s_i_flux}$$

<a id='define'></a>

### Step 1.a: The function declaration \[Back to [top](#toc)\]
$$\label{define}$$

First, we give the function definition. This function will take as parameters the integers `i`, `j`, and `k`; these indicate a point on the grid, but appear not to be used. The function also needs the parameter `flux_dirn` to know which direction to compute the flux in, the face values `Ul` and `Ur` (see [the previous tutorial](Tutorial-GiRaFFE_HO_Ccode_library-PPM.ipynb)), the interpolated quantities on the cell interfaces `FACEVAL` and `FACEVAL_LAPSE_PSI4`, the speeds `cmax` and `cmin`, and the fluxes `st_x_flux`, `st_y_flux`, and `st_z_flux`. We also calculate quantities related to the conformal factor $\psi$ (including the square root of the determinant of the metric $\sqrt{\gamma}$) and the lapse $\alpha$; these lines will need to be modified to use the values actually stored by our `GiRaFFE`.

In [ ]:
%%writefile $outdir/GRFFE__S_i__flux.C
//-----------------------------------------------------------------------------
// Compute the flux for advecting S_i .
//-----------------------------------------------------------------------------
static inline void GRFFE__S_i__flux(const int i,const int j,const int k,const int flux_dirn, REAL *Ul, REAL *Ur, REAL *FACEVAL,const REAL *FACEVAL_LAPSE_PSI4,
                                    REAL &cmax,REAL &cmin,
                                    REAL &st_x_flux,REAL &st_y_flux,REAL &st_z_flux) {


In [ ]:
# Conveniently, we store the metric determinant gamma directly.
alpha_sqrt_gamma = alpha*sp.sqrt(gammadet)

# We'll also compute some powers of the conformal factor psi:
# Since psi^6 = gammadet,
psi = gammadet**(1.0/6.0)
psim4 = 1.0/(psi**4.0)

In [ ]:
# OLD:
#   const REAL psi4 = FACEVAL_LAPSE_PSI4[PSI4];
#   const REAL psi6 = FACEVAL_LAPSE_PSI4[PSI4]*FACEVAL_LAPSE_PSI4[PSI2];
#   const REAL psim4 = 1.0/(psi4);

#   const REAL alpha_sqrt_gamma = FACEVAL_LAPSE_PSI4[LAPSE]*psi6;
#   const REAL ONE_OVER_LAPSE = 1.0/FACEVAL_LAPSE_PSI4[LAPSE];
#   const REAL ONE_OVER_LAPSE_SQUARED=SQR(ONE_OVER_LAPSE);

<a id='speed_limit'></a>

### Step 1.b: Speed-limit the velocities \[Back to [top](#toc)\]
$$\label{speed_limit}$$

Next, we compute the speed-limited velocities on the faces. This function is found in `inlined_functions.C`, and will be documented elsewhere **(TODO: link)**. This function also counts how often the speed had to be limited.

In [ ]:
%%writefile -a $outdir/GRFFE__S_i__flux.C

  //Compute face velocities
  // Begin by computing u0
  output_stats stats; stats.failure_checker=0;
  REAL u0_r,u0_l;
  impose_speed_limit_output_u0(FACEVAL,Ur,psi4,ONE_OVER_LAPSE,stats,u0_r);
  impose_speed_limit_output_u0(FACEVAL,Ul,psi4,ONE_OVER_LAPSE,stats,u0_l);


In [ ]:
# We shouldn't have to do anything here? Or we need to rerun our function on 
# the reconstructed data. Is that necessary?

<a id='smallb'></a>

### Step 1.c: Magnetic field in the comoving fluid frame \[Back to [top](#toc)\]
$$\label{smallb}$$

Now, we will compute the magnetic field in the comoving fluid frame $b^\mu$ , as defined in [Gammie's paper](https://arxiv.org/pdf/astro-ph/0301509.pdf).  We will also compute the four velocities on the faces $u_\mu$.

In [ ]:
%%writefile -a $outdir/GRFFE__S_i__flux.C

  //Next compute b^{\mu}, the magnetic field measured in the comoving fluid frame:
  const REAL ONE_OVER_LAPSE_SQRT_4PI = ONE_OVER_LAPSE*ONE_OVER_SQRT_4PI;
  /***********************************************************/
  /********** RIGHT FACE ************/
  // Note that smallbr[4] = b^a defined in Gammie's paper, on the right face.
  REAL u_x_over_u0_psi4r,u_y_over_u0_psi4r,u_z_over_u0_psi4r;
  REAL smallbr[NUMVARS_SMALLB];
  // Compute b^{a}, b^2, and u_i over u^0
  compute_smallba_b2_and_u_i_over_u0_psi4(FACEVAL,FACEVAL_LAPSE_PSI4,Ur,u0_r,ONE_OVER_LAPSE_SQRT_4PI,  
                                          u_x_over_u0_psi4r,u_y_over_u0_psi4r,u_z_over_u0_psi4r,smallbr);
  // Then compute u_xr,u_yr, and u_zr. We need to set the zeroth component so we can specify U_LOWER{r,l}[{UX,UY,UZ}] (UX=1,UY=2,UZ=3).
  const REAL U_LOWERr[4] = { 0.0, u_x_over_u0_psi4r*u0_r*FACEVAL_LAPSE_PSI4[PSI4], u_y_over_u0_psi4r*u0_r*FACEVAL_LAPSE_PSI4[PSI4], 
                                  u_z_over_u0_psi4r*u0_r*FACEVAL_LAPSE_PSI4[PSI4] };
  /********** LEFT FACE ************/
  // Note that smallbl[4] = b^a defined in Gammie's paper, on the left face.
  REAL u_x_over_u0_psi4l,u_y_over_u0_psi4l,u_z_over_u0_psi4l;
  REAL smallbl[NUMVARS_SMALLB];
  // Compute b^{a}, b^2, and u_i over u^0
  compute_smallba_b2_and_u_i_over_u0_psi4(FACEVAL,FACEVAL_LAPSE_PSI4,Ul,u0_l,ONE_OVER_LAPSE_SQRT_4PI,  
                                          u_x_over_u0_psi4l,u_y_over_u0_psi4l,u_z_over_u0_psi4l,smallbl);
  // Then compute u_xr,u_yr, and u_zr. We need to set the zeroth component so we can specify U_LOWER{r,l}[{UX,UY,UZ}]
  const REAL U_LOWERl[4] = { 0.0, u_x_over_u0_psi4l*u0_l*FACEVAL_LAPSE_PSI4[PSI4], u_y_over_u0_psi4l*u0_l*FACEVAL_LAPSE_PSI4[PSI4], 
                                  u_z_over_u0_psi4l*u0_l*FACEVAL_LAPSE_PSI4[PSI4] };
  /***********************************************************/


In [ ]:
# We already did something like this in GiRaFFE_Higher_Order_v2; we'll 
# paste that code here, but using the face variables.
import u0_smallb_Poynting__Cartesian.u0_smallb_Poynting__Cartesian as u0b

# LEFT
u0b.compute_u0_smallb_Poynting__Cartesian(gammaDD,betaU,alpha,Valenciav_lU,B_lU)
u_lD = ixp.zerorank1()
u_lU = ixp.zerorank1()
u4upperZero_l = gri.register_gridfunctions("AUX","u4upperZero_l")

for i in range(DIM):
    u_lD[i] = u0b.u_lD[i].subs(u0b.u0,u4upperZero_l)
    u_lU[i] = u0b.u_lU[i].subs(u0b.u0,u4upperZero_l)

smallb4_lU = ixp.zerorank1(DIM=4)
smallb4_lD = ixp.zerorank1(DIM=4)
for mu in range(4):
    smallb4_lU[mu] = u0b.smallb4_lU[mu].subs(u0b.u0,u4upperZero_l)
    smallb4_lD[mu] = u0b.smallb4_lD[mu].subs(u0b.u0,u4upperZero_l)

smallb2 = u0b.smallb2etk.subs(u0b.u0,u4upperZero_l)

# RIGHT
u0b.compute_u0_smallb_Poynting__Cartesian(gammaDD,betaU,alpha,Valenciav_rU,B_rU)
u_rD = ixp.zerorank1()
u_rU = ixp.zerorank1()
u4upperZero_r = gri.register_gridfunctions("AUX","u4upperZero_r")

for i in range(DIM):
    u_rD[i] = u0b.u_rD[i].subs(u0b.u0,u4upperZero_r)
    u_rU[i] = u0b.u_rU[i].subs(u0b.u0,u4upperZero_r)

smallb4_rU = ixp.zerorank1(DIM=4)
smallb4_rD = ixp.zerorank1(DIM=4)
for mu in range(4):
    smallb4_rU[mu] = u0b.smallb4_rU[mu].subs(u0b.u0,u4upperZero_r)
    smallb4_rD[mu] = u0b.smallb4_rD[mu].subs(u0b.u0,u4upperZero_r)

smallb2 = u0b.smallb2etk.subs(u0b.u0,u4upperZero_r)


<a id='hydro_speed'></a>

### Step 1.d: Hydrodynamic wave speeds \[Back to [top](#toc)\]
$$\label{hydro_speed}$$

Next, we will find the speeds at which the hydrodynamics waves propagate. We start from the speed of light (since FFE deals with very diffuse plasmas), which is $c=1.0$ in our chosen units. We then find the speeds $c_+$ and $c_-$ on each face with the function `find_cp_cm` **(TODO: link to documentation)**; then, we find minimum and maximum speeds possible from among those.

In [ ]:
%%writefile -a $outdir/GRFFE__S_i__flux.C

  // v02 is the speed of light in FFE, which is 1.0 in G=c=1.0.
  const REAL v02r=1.0,v02l=1.0;
  
  int offset=flux_dirn-1;

  REAL cplusr,cminusr,cplusl,cminusl;
  find_cp_cm(cplusr,cminusr,v02r,u0_r,
             Ur[VX+offset],ONE_OVER_LAPSE_SQUARED,FACEVAL[SHIFTX+offset],psim4,FACEVAL[GUPXX+offset]);
  find_cp_cm(cplusl,cminusl,v02l,u0_l,
             Ul[VX+offset],ONE_OVER_LAPSE_SQUARED,FACEVAL[SHIFTX+offset],psim4,FACEVAL[GUPXX+offset]);

  // Then compute cmax, cmin. This is required for the HLL flux.
  const REAL cmaxL =  MAX(0.0,MAX(cplusl,cplusr));
  const REAL cminL = -MIN(0.0,MIN(cminusl,cminusr));



<a id='useful_quantities'></a>

### Step 1.e: Some final needed quantities \[Back to [top](#toc)\]
$$\label{useful_quantities}$$

Now, we lower the index on the spatial parts of $b^\mu$, $b^i$. Since $b_\nu = g_{\mu\nu} b^\mu$ and 
$$
g_{\mu\nu} = \begin{pmatrix} 
-\alpha^2 + \beta^k \beta_k & \beta_j \\
\beta_i & \gamma_{ij}
\end{pmatrix},
$$
it follows that $b_j = g_{ij}(b^i + b^t \beta^i)$. We calculate this on both the left and right face of the cell.

In [ ]:
%%writefile -a $outdir/GRFFE__S_i__flux.C

  //*********************************************************************
  // momentum flux = \alpha \sqrt{\gamma} T^m_j, where m is the current flux direction (the m index)
  //*********************************************************************
  // b_j = g_{ij} (b^i + b^t shift^i), g_{ij} = physical metric
  //REAL sbtr=0,sbtl=0;
  REAL smallb_lowerr[NUMVARS_SMALLB],smallb_lowerl[NUMVARS_SMALLB];
  lower_4vector_output_spatial_part(psi4,FACEVAL,smallbr,smallb_lowerr);
  lower_4vector_output_spatial_part(psi4,FACEVAL,smallbl,smallb_lowerl);


We will also compute some useful hydrodynamics quantities on each face. These are simplified by the fact that pressure and density are both zero in FFE: 
\begin{align}
\rho + b^2 &= b^2 \\
P + \frac{1}{2} b^2 &= \frac{1}{2} b^2\\
\end{align}

In [ ]:
%%writefile -a $outdir/GRFFE__S_i__flux.C

  /********** Flux for S_x **********/
  // [S_x flux] = \alpha \sqrt{\gamma} T^m_x, where m is the current flux direction (the m index)
  //    Again, offset = 0 for reconstruction in x direction, 1 for y, and 2 for z
  //    Note that kronecker_delta[flux_dirn][0] = { 1 if flux_dirn==1, 0 otherwise }.
  /********** RIGHT FACE ************/
  // Compute a couple useful hydro quantities:
  const REAL rho0_h_plus_b2_r = smallbr[SMALLB2];     // Since rho=0 in GRFFE
  const REAL P_plus_half_b2_r = 0.5*smallbr[SMALLB2]; // Since P=0 in GRFFE
  /********** LEFT FACE *************/
  // Compute a couple useful hydro quantities:
  const REAL rho0_h_plus_b2_l = smallbl[SMALLB2];     // Since rho=0 in GRFFE
  const REAL P_plus_half_b2_l = 0.5*smallbl[SMALLB2]; // Since P=0 in GRFFE


<a id='fluxes'></a>

### Step 1.f: Compute the fluxes \[Back to [top](#toc)\]
$$\label{fluxes}$$

Finally, we can compute the flux in each direction. This momentum flux in the $m$ direction is defined as $\alpha \sqrt{\gamma} T^m_{\ \ j}$. We have already defined $\alpha \sqrt{\gamma}$, so all we need to do is calculate $T^m_{\ \ j}$. We will do so in accordance with the method published by [Harten, Lax, and von Leer](https://epubs.siam.org/doi/pdf/10.1137/1025002) (hereafter HLL) to solve the Riemann problem. So, we define $f(u) = T^m_{\ \ j}$ on each face as 
$$
f = \alpha \sqrt{\gamma} \left( (\rho+b^2)(u^0 v^j) v_x + (P+\frac{1}{2}b^2) \delta_{jx} - b^j b_x \right);
$$
we use $j$ to correspond to the direction in which we are calculating the flux, based on the input `flux_dirn`; that is, $j=1$ corresponds to $x$, and so forth. $\delta_{ij}$ is the standard Kronecker delta. We also define `st_x_r` and `st_x_l` (called $u_{\rm R}$ and $u_{\rm L}$ in HLL; we'll not do that, since we're using $u$ for four-velocity):
$$
{\rm st\_x} = \alpha \sqrt{\gamma} \left( (\rho+b^2) u^0 v^j v_x - b^t b_x \right)
$$
and combine based on eq. 3.15 in the HLL paper.

In [ ]:
%%writefile -a $outdir/GRFFE__S_i__flux.C

  REAL Fr = alpha_sqrt_gamma*( rho0_h_plus_b2_r*(u0_r*Ur[VX+offset])*U_LOWERr[UX] 
                                    + P_plus_half_b2_r*kronecker_delta[flux_dirn][0] - smallbr[SMALLBX+offset]*smallb_lowerr[SMALLBX] );
  REAL Fl = alpha_sqrt_gamma*( rho0_h_plus_b2_l*(u0_l*Ul[VX+offset])*U_LOWERl[UX] 
                                    + P_plus_half_b2_l*kronecker_delta[flux_dirn][0] - smallbl[SMALLBX+offset]*smallb_lowerl[SMALLBX] );

  //        S_x =\alpha\sqrt{\gamma}( T^0_x )
  const REAL st_x_r = alpha_sqrt_gamma*( rho0_h_plus_b2_r*u0_r*U_LOWERr[UX] - smallbr[SMALLBT]*smallb_lowerr[SMALLBX] );
  const REAL st_x_l = alpha_sqrt_gamma*( rho0_h_plus_b2_l*u0_l*U_LOWERl[UX] - smallbl[SMALLBT]*smallb_lowerl[SMALLBX] );

  // HLL step for Sx:
  st_x_flux = (cminL*Fr + cmaxL*Fl - cminL*cmaxL*(st_x_r-st_x_l) )/(cmaxL + cminL);
%%writefile -a $outdir/GRFFE__S_i__flux.C


We repeat this step in the $y$ direction:
$$
f = \alpha \sqrt{\gamma} \left( (\rho+b^2)(u^0 v^j) v_y + (P+\frac{1}{2}b^2) \delta_{jy} - b^j b_y \right);
$$
and 
$$
{\rm st\_y} = \alpha \sqrt{\gamma} \left( (\rho+b^2) u^0 v^j v_y - b^t b_y \right),
$$

In [ ]:
%%writefile -a $outdir/GRFFE__S_i__flux.C

  /********** Flux for S_y **********/
  // [S_y flux] = \alpha \sqrt{\gamma} T^m_y, where m is the current flux direction (the m index)
  //    Again, offset = 1 for reconstruction in x direction, 2 for y, and 3 for z
  //    Note that kronecker_delta[flux_dirn][1] = { 1 if flux_dirn==2, 0 otherwise }.
  Fr = alpha_sqrt_gamma*( rho0_h_plus_b2_r*(u0_r*Ur[VX+offset])*U_LOWERr[UY] + P_plus_half_b2_r*kronecker_delta[flux_dirn][1] 
                          - smallbr[SMALLBX+offset]*smallb_lowerr[SMALLBY] );
  Fl = alpha_sqrt_gamma*( rho0_h_plus_b2_l*(u0_l*Ul[VX+offset])*U_LOWERl[UY] + P_plus_half_b2_l*kronecker_delta[flux_dirn][1] 
                          - smallbl[SMALLBX+offset]*smallb_lowerl[SMALLBY] );

  //        S_y =\alpha\sqrt{\gamma}( T^0_y )
  const REAL st_y_r = alpha_sqrt_gamma*( rho0_h_plus_b2_r*u0_r*U_LOWERr[UY] - smallbr[SMALLBT]*smallb_lowerr[SMALLBY] );
  const REAL st_y_l = alpha_sqrt_gamma*( rho0_h_plus_b2_l*u0_l*U_LOWERl[UY] - smallbl[SMALLBT]*smallb_lowerl[SMALLBY] );

  // HLL step for Sy:
  st_y_flux = (cminL*Fr + cmaxL*Fl - cminL*cmaxL*(st_y_r-st_y_l) )/(cmaxL + cminL);


and in z:
$$
f = \alpha \sqrt{\gamma} \left( (\rho+b^2)(u^0 v^j) v_z + (P+\frac{1}{2}b^2) \delta_{jz} - b^j b_z \right);
$$
and 
$$
{\rm st\_z} = \alpha \sqrt{\gamma} \left( (\rho+b^2) u^0 v^j v_z - b^t b_z \right)
$$

In [ ]:
%%writefile -a $outdir/GRFFE__S_i__flux.C

  /********** Flux for S_z **********/
  // [S_z flux] = \alpha \sqrt{\gamma} T^m_z, where m is the current flux direction (the m index)
  //    Again, offset = 1 for reconstruction in x direction, 2 for y, and 3 for z
  //    Note that kronecker_delta[flux_dirn][2] = { 1 if flux_dirn==3, 0 otherwise }.
  Fr = alpha_sqrt_gamma*( rho0_h_plus_b2_r*(u0_r*Ur[VX+offset])*U_LOWERr[UZ] + P_plus_half_b2_r*kronecker_delta[flux_dirn][2] 
                          - smallbr[SMALLBX+offset]*smallb_lowerr[SMALLBZ] );
  Fl = alpha_sqrt_gamma*( rho0_h_plus_b2_l*(u0_l*Ul[VX+offset])*U_LOWERl[UZ] + P_plus_half_b2_l*kronecker_delta[flux_dirn][2] 
                          - smallbl[SMALLBX+offset]*smallb_lowerl[SMALLBZ] );

  //        S_z =\alpha\sqrt{\gamma}( T^0_z )
  const REAL st_z_r = alpha_sqrt_gamma*( rho0_h_plus_b2_r*u0_r*U_LOWERr[UZ] - smallbr[SMALLBT]*smallb_lowerr[SMALLBZ] );
  const REAL st_z_l = alpha_sqrt_gamma*( rho0_h_plus_b2_l*u0_l*U_LOWERl[UZ] - smallbl[SMALLBT]*smallb_lowerl[SMALLBZ] );

  // HLL step for Sz:
  st_z_flux = (cminL*Fr + cmaxL*Fl - cminL*cmaxL*(st_z_r-st_z_l) )/(cmaxL + cminL);

  cmax = cmaxL;
  cmin = cminL;
}


<a id='latex_pdf_output'></a>

# Step 2: Output this module to $\LaTeX$-formatted PDF file \[Back to [top](#toc)\]
$$\label{latex_pdf_output}$$

The following code cell converts this Jupyter notebook into a proper, clickable $\LaTeX$-formatted PDF file. After the cell is successfully run, the generated PDF may be found in the root NRPy+ tutorial directory, with filename
[Tutorial-GiRaFFE_HO_Ccode_library-Stilde-flux.pdf](Tutorial-GiRaFFE_HO_Ccode_library-Stilde-flux.pdf) (Note that clicking on this link may not work; you may need to open the PDF file through another means.)

In [ ]:
!jupyter nbconvert --to latex --template latex_nrpy_style.tplx Tutorial-GiRaFFE_HO_Ccode_library-Stilde-flux.ipynb
!pdflatex -interaction=batchmode Tutorial-GiRaFFE_HO_Ccode_library-Stilde-flux.tex
!pdflatex -interaction=batchmode Tutorial-GiRaFFE_HO_Ccode_library-Stilde-flux.tex
!pdflatex -interaction=batchmode Tutorial-GiRaFFE_HO_Ccode_library-Stilde-flux.tex
!rm -f Tut*.out Tut*.aux Tut*.log